In [1]:
import os
os.environ['VLLM_USE_V1'] = '1' # this should be done befre any other import

import torch
import torch.nn as nn
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from vllm.sampling_params import GuidedDecodingParams

/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 10-21 17:32:35 [__init__.py:216] Automatically detected platform cuda.


In [2]:
# standard python libraries
from pathlib import Path

from typing import Dict, List, Union, Optional, Tuple
from tqdm import tqdm
import json
import os
import sys

# Data Science librraies
import numpy as np
import pandas as pd

from pprint import pprint

# Pytorch
import torch

# Huggingface Librraies
import evaluate
from datasets import load_dataset

from huggingface_hub import login

# Logging and secrets
import wandb
from huggingface_hub import login, HfApi, create_repo

/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may

In [3]:
sys.path

['/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python310.zip',
 '/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10',
 '/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/lib-dynload',
 '',
 '/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages',
 '/tmp/tmpalea_buh']

In [ ]:
#Wandb & Huggingface keys
hf_token = "***REMOVED***"
wandb_api_key = "***REMOVED***"

In [5]:
# Intitialize Weights & Biases
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to WANDB!")
else:
    print("No wandb key provided. Skipping wandb login.")

if hf_token:
    
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/012/r/rx/rxh210037/.netrc
wandb: Currently logged in as: reyhaneh-rhp7 (reyhaneh-rhp7-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!
Successfully logged in to Hugging Face!


In [6]:
wandb.init(project="PartC_Inference_vLLM", name="PartC_Inference_vLLM")  


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
data_folder = Path("***/Part C/jigsaw-agile-community-rules")
train_Path = data_folder / "train.csv"
test_Path = data_folder / "test.csv"
train = pd.read_csv(train_Path)
test = pd.read_csv(test_Path)
train.shape, test.shape

((2029, 9), (10, 8))

In [8]:

dataset_prepared_path = "./prepared_data/"
output_dir = "./outputs_qwen_ruleviolation/"
wandb_project = "PartC_Inference_vLLM"
base_model = "Qwen/Qwen1.5-7B"
# hf_profile = "your_hf_username"


In [9]:
BASE_PROMPT = (
    "You are a rule compliance analyst. "
    "Your task is to determine whether the following Reddit comment complies with the subreddit rule or violates it. "
    "Only respond with 'complies' or 'violates'.\n\n"
)
COMPLETE_PHRASE = "Respond with only one word: 'complies' or 'violates'."

# Build the full user prompt per example
def build_prompt(row):
    return f"""
{BASE_PROMPT}
Subreddit: r/{row['subreddit']}
Rule: {row['rule']}

Positive Examples:
1) {row['positive_example_1']}
{COMPLETE_PHRASE}

2) {row['positive_example_2']}
{COMPLETE_PHRASE}

Negative Examples:
1) {row['negative_example_1']}
{COMPLETE_PHRASE}

2) {row['negative_example_2']}
{COMPLETE_PHRASE}

---
Comment: {row['body']}
{COMPLETE_PHRASE}
""".strip()

In [10]:
# Build inference format (no assistant replies)
def df_to_inference_format(df):
    return df.apply(
        lambda r: {
            "messages": [
                {"role": "user", "content": build_prompt(r)}
            ]
        },
        axis=1
    ).tolist()

In [11]:
import json
output_dir = Path("prepared_data")
output_dir.mkdir(exist_ok=True)

test_inference_data = df_to_inference_format(test)
# Save to JSONL
def save_jsonl(data, filename):
    path = output_dir / filename
    with open(path, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item) + "\n")
    print(f" Saved {len(data)} records to {path}")


save_jsonl(test_inference_data, "test_inference.jsonl")

 Saved 10 records to prepared_data/test_inference.jsonl


In [12]:
TEST_PATH = "./prepared_data/test_inference.jsonl"       # or test file path
LORA_ADAPTER_PATH = "./outputs_qwen_ruleviolation/checkpoint-363"  
BASE_MODEL  = "Qwen/Qwen1.5-7B"


In [13]:

from vllm import LLM, SamplingParams
# Sampling / decoding parameters
sampling_params = SamplingParams(
    temperature=0.0,
    max_tokens=8,
    top_p=1.0
)


In [14]:
llm = LLM(
    model=BASE_MODEL,
    dtype="bfloat16",              # string, not torch dtype
    trust_remote_code=True,
    enable_lora=True,
    max_lora_rank=128,
    gpu_memory_utilization=0.6,
    max_model_len=1024,             # or larger if you have VRAM
    guided_decoding_backend="outlines",
)

INFO 10-21 17:32:39 [utils.py:233] non-default args: {'trust_remote_code': True, 'dtype': 'bfloat16', 'max_model_len': 1024, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'enable_lora': True, 'max_lora_rank': 128, 'guided_decoding_backend': 'outlines', 'model': 'Qwen/Qwen1.5-7B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-21 17:32:40 [model.py:547] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-21 17:32:40 [model.py:1510] Using max model len 1024


...

INFO 10-21 17:32:40 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-21 17:32:40 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
WARNING 10-21 17:32:40 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-21 17:32:43 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:44 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:44 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen1.5-7B', speculative_config=None, tokenizer='Qwen/Qwen1.5-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokeniz

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.74it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.45it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.34it/s]
(EngineCore_DP0 pid=516788) 


(EngineCore_DP0 pid=516788) INFO 10-21 17:32:47 [default_loader.py:267] Loading weights took 1.75 seconds
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:47 [punica_selector.py:19] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:48 [gpu_model_runner.py:2653] Model loading took 14.9880 GiB and 2.198537 seconds
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:53 [backends.py:548] Using cache directory: /home/012/r/rx/rxh210037/.cache/vllm/torch_compile_cache/e4e93bc2fa/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:53 [backends.py:559] Dynamo bytecode transform time: 5.50 s
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:55 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.545 s
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:57 [monitor.py:34] torch.compile takes 5.50 s in total
(EngineCore_DP0 pid=516788) INFO 10-21 17:32:58 [gpu_worker.py:298] Available KV cache memory: 11.97 GiB
(EngineCore_

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:05<00:00, 11.18it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:02<00:00, 12.79it/s]


(EngineCore_DP0 pid=516788) INFO 10-21 17:33:08 [gpu_model_runner.py:3480] Graph capturing finished in 9 secs, took 0.96 GiB
(EngineCore_DP0 pid=516788) INFO 10-21 17:33:08 [core.py:210] init engine (profile, create kv cache, warmup model) took 19.92 seconds
INFO 10-21 17:33:08 [llm.py:306] Supported_tasks: ['generate']


In [15]:
from vllm.lora.request import LoRARequest
# Apply the trained LoRA adapter
lora_request = LoRARequest(
    lora_name="PartC_Inference",
    lora_int_id=1,
    lora_path=LORA_ADAPTER_PATH
)


In [16]:
import re

def normalize_prediction(text):
    text = text.lower().strip()
    if "vio" in text:
        return "violates"
    elif "com" in text:
        return "complies"
    else:
        return "unknown"



In [17]:
prompts = []
with open(TEST_PATH, "r", encoding="utf-8") as f:
    for line in f:
        ex = json.loads(line)
        user_msg = next(m["content"] for m in ex["messages"] if m["role"] == "user")
        prompts.append(user_msg)

print(f"✅ Loaded {len(prompts)} examples for inference")


✅ Loaded 10 examples for inference


In [23]:
from tqdm import tqdm
predictions = []
batch_size = 16

for i in tqdm(range(0, len(prompts), batch_size), desc="Generating predictions"):
    batch = prompts[i : i + batch_size]
    outputs = llm.generate(batch, sampling_params, lora_request=lora_request)
    for o in outputs:
        raw = o.outputs[0].text.strip()
        predictions.append(normalize_prediction(raw))
test['predicted_label'] = predictions
print(f"✅ Generated {len(predictions)} predictions")

Generating predictions: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s]

✅ Generated 10 predictions


In [19]:
from collections import Counter
counts = Counter(predictions)
print("\n🔍 Prediction Summary:")
for k, v in counts.items():
    print(f"{k:10s} : {v}")



🔍 Prediction Summary:
complies   : 5
violates   : 5


In [20]:
raw_texts = []     # store the actual LLM outputs
predictions = []   # normalized labels

for i in tqdm(range(0, len(prompts), batch_size), desc="Generating predictions"):
    batch = prompts[i : i + batch_size]
    outputs = llm.generate(batch, sampling_params, lora_request=lora_request)
    for o in outputs:
        raw = o.outputs[0].text.strip()
        raw_texts.append(raw)
        predictions.append(normalize_prediction(raw))


Generating predictions: 100%|██████████| 1/1 [00:00<00:00,  3.37it/s]


In [24]:
test['predicted_label'] = test['predicted_label'].map({'complies': 0, 'violates': 1})
test

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,predicted_label
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...,0
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...,0
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...,1
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...,1
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...,1
5,2034,Top 10 Pentakills in August https://youtu.be/Z...,"No Advertising: Spam, referral links, unsolici...",leagueoflegends,code free tyrande --->>> [Imgur](http://i.imgu...,see tits sweet at this point http://sh.or...,its my asreddit https://www.youtube.com/watch?...,click here for more videos[Russian mafia messe...,0
6,2035,WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...,"No Advertising: Spam, referral links, unsolici...",AskReddit,"[ENJOY 50,000 BONUS POINTS after $4,000 spent ...",We have high quality Canadian Polymer banknote...,i was on the same ios and try this jailbreak m...,We're streaming Pokemon Veitnamese Crystal RIG...,1
7,2036,[This mixtape is lit FAM!](https://beholdthear...,"No Advertising: Spam, referral links, unsolici...",BlackPeopleTwitter,watch good one hooters there http://https://x5...,bad pussies waiting for you herein http://gour...,[Full HD Movie Online Free](http://www.flickma...,* Computers :http://livematchstreamz.blogspot....,0
8,2037,must be watch movie https://sites.google.com/s...,"No Advertising: Spam, referral links, unsolici...",movies,see there hooters sweet http://url.t.aylo...,"Sign up with my link and get $1,000 of free pr...",The fact that there are pro and cons to everyt...,Oh that's interesting. You should check out my...,0
9,2038,Free paypal cards here!! https://www.pointspri...,"No Advertising: Spam, referral links, unsolici...",pics,she will come your home open her legs with an...,Old and young your girl-next-door make sex wi...,"i think the conjuring 2 is not bad movie ,The...","wow, she's so beautyful\n>[Sakura Vietnam](htt...",1


In [26]:
#  Save predictions
output_path = "./inference_predictions.csv"
test.to_csv(output_path, index=False)
print(f"Saved predictions to {output_path}")

Saved predictions to ./inference_predictions.csv


In [27]:
# Finish wandb run
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.
